In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N, K):                                 ###data为原始数据，stock_eg为全数据股票，T为所要比较的涨跌幅的日期，N为T之前的时间跨度，K为最大成交总额及涨跌幅对比数

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N-1)-stock_eg.index[0]      ###寻找T前N+1天的索引位置
    end = int(stock_eg[stock_eg['date'] ==  T].index.values)-stock_eg.index[0]             ###寻找T日的索引位置
    slc = stock_eg.iloc[[start,end-1,end],:]
    
    init = data[data['date'] == slc['date'].values[0]]
    fina = data[data['date'] == slc['date'].values[1]]
    fina1 = data[data['date'] == slc['date'].values[2]]
    secl_sum = init.append(fina).sort_index()
    secl_sum = secl_sum.append(fina1).sort_index()                    ####将所有在T,T-N-1,T-1时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(fina['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina1['stock'])))    ###选出在上述三个时段都有数据的股票
    
    starter = secl_sum[secl_sum['stock'] == stock_intersect[0]]
    turn_o = starter['turnover_sum']
    differ = turn_o.values[1]-turn_o.values[0]
    stock_f = starter[starter['date'] == T]
    stock_f.insert(5,'turnover_N',[differ])                          ###将上述选出的股票计算N天的成交量总额及T日的涨跌幅
    
    for k in stock_intersect[1:]:
        a = secl_sum[secl_sum['stock'] == k]
        turn_o = a['turnover_sum']
        differ = turn_o.values[1]-turn_o.values[0]
        b = a[a['date'] == T]
        b.insert(5,'turnover_N',[differ])
        stock_f = stock_f.append(b)
    stock_f = stock_f.sort_index()                                 ###循环进行上述操作
    
    stock_Tmax = stock_f.sort_values(by = ['turnover_N'], ascending = False).head(K)     ###取出N天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    

In [4]:
def statisc(data, stock_eg, N, K):

    data = data[['date','stock','close','turnover_sum','change']]
    data['stock'] = data['stock'].astype(str)              ###更改stock的数据类型为字符型
    
    statis = pd.DataFrame(columns = ['date', 'max_change', 'max_stock', 'count'])

    for i in stock_eg['date'].values[N+1:]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N, K)
            size = stock_bestinf.iloc[:,0].size         ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
            max_change = max(stock_bestinf['change'])
            max_stockinf = stock_bestinf[stock_bestinf['change'] == max_change]    ###找出之中涨幅最大的股票信息
            max_stock = list(max_stockinf['stock'].values)
            new_data = {'date':stock_bestinf['date'].values[0], 'max_change':max_change, 'max_stock':max_stock, 'count':size}
            new_add = pd.DataFrame(new_data, columns = ['date', 'max_change', 'max_stock', 'count'])     ###将列表合并
            statis = statis.append(new_add, ignore_index=True)
        except:
            continue

    return statis

In [5]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2015/2015.csv',encoding='gbk',engine='python')

In [6]:
stock_m = max_stock(data)

stock_eg = data[data['stock'] == stock_m]

#stock_bestinf = best_perf(data, stock_eg, '2017-02-08', 20, 100)

In [11]:
statis = statisc(data, stock_eg, 20, 100)
statis

D:\Anaconda3\envs\dayu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,date,max_change,max_stock,count
0,2015/2/3,0.099944,2183,11
1,2015/2/4,0.099783,600528,3
2,2015/2/5,0.099926,300104,5
3,2015/2/6,0.046312,601555,2
4,2015/2/9,0.100172,2673,18
5,2015/2/10,0.100353,600528,8
6,2015/2/11,0.100047,600570,7
7,2015/2/12,0.099957,600352,7
8,2015/2/13,0.100071,300104,5
9,2015/2/16,0.101031,600050,12


In [12]:
statis.to_csv("D:/Documents/Go Abroad/实习/成交金额/Python数据/2015/2015-20.csv", index=False)

In [ ]:
data

In [ ]:
statis = pd.DataFrame(columns = ['date', 'max_change', 'max_stock', 'count'])
for i in stock_eg['date'].values[154+1:160]:
    try:
        stock_bestinf = best_perf(data, stock_eg, i, N, K)
        size = stock_bestinf.iloc[:,0].size
        max_change = max(stock_bestinf['change'])
        max_stockinf = stock_bestinf[stock_bestinf['change'] == max_change]
        max_stock = list(max_stockinf['stock'].values)
        new_data = {'date':stock_bestinf['date'].values[0], 'max_change':max_change, 'max_stock':max_stock, 'count':size}
        new_add = pd.DataFrame(new_data, columns = ['date', 'max_change', 'max_stock', 'count'])
        statis = statis.append(new_add, ignore_index=True)
    except:
        continue

In [ ]:
statis

In [ ]:
stock_eg['date'].values[156]